In [1]:
import pandas as pd
import numpy as np

import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot
init_notebook_mode(connected=True)

dateCutoff = pd.Timestamp.today().normalize() - pd.Timedelta(days=22)

###### WHO Data by smbd
urlWHO = 'https://raw.github.com/eebrown/data2019nCoV/master/data-raw/WHO_SR.csv'
dfWHO = pd.read_csv(urlWHO, error_bad_lines=False)

dfWHO['Date'] = pd.to_datetime(dfWHO['Date'], format='%Y-%m-%d')
dfWHO.replace(['High', 'Very High'], [0, 0], inplace=True)

dfWHO = dfWHO.melt(id_vars=['SituationReport', 'Date'], 
        var_name='Area', 
        value_name='Value')



dfWHO.fillna(0, inplace=True)
dfWHO['Value'] = pd.to_numeric(dfWHO['Value'], downcast='integer')

dfWHO.sort_values(by=['Area', 'Date'], inplace=True)
dfWHO = dfWHO[dfWHO['Date'] > dateCutoff]

dfWHO.set_index(['Area', 'Date'],inplace =True)

covidIncWHO = dfWHO.groupby(level=['Area'])['Value'].diff().reset_index()


###### JHU Data detailed
urlCSSE = 'https://raw.github.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv'
dfCSSE = pd.read_csv(urlCSSE, error_bad_lines=False)
dfCSSE = dfCSSE.melt(id_vars=['Country/Region', 'Province/State', 'Lat', 'Long'], 
        var_name='Date', 
        value_name='Value')

dfCSSE['Date'] = pd.to_datetime(dfCSSE['Date'], format='%m/%d/%y')

dfCSSE.fillna('no data', inplace=True)
dfCSSE.sort_values(by=['Country/Region', 'Province/State', 'Date'], inplace=True)
dfCSSE = dfCSSE[dfCSSE['Date'] > dateCutoff]

dfCSSE.set_index(['Country/Region', 'Province/State', 'Date'],inplace =True)

covidIncCSSE = dfCSSE.groupby(level=['Country/Region', 'Province/State'])['Value'].diff().reset_index()
covidIncByCountryCSSE = covidIncCSSE.groupby(['Country/Region', 'Date'])['Value'].sum().reset_index()


###### JHU Data global
urlCSSEG = 'https://raw.github.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
dfCSSEG = pd.read_csv(urlCSSEG, error_bad_lines=False)
dfCSSEG = dfCSSEG.melt(id_vars=['Country/Region', 'Province/State', 'Lat', 'Long'], 
        var_name='Date', 
        value_name='Value')

dfCSSEG['Date'] = pd.to_datetime(dfCSSEG['Date'], format='%m/%d/%y')

dfCSSEG.fillna('no data', inplace=True)
dfCSSEG.sort_values(by=['Country/Region', 'Province/State', 'Date'], inplace=True)
dfCSSEG = dfCSSEG[dfCSSEG['Date'] > dateCutoff]

dfCSSEG.set_index(['Country/Region', 'Province/State', 'Date'],inplace =True)

covidIncCSSEG = dfCSSEG.groupby(level=['Country/Region', 'Province/State'])['Value'].diff().reset_index()
covidIncAllCSSEG = covidIncCSSEG.groupby(['Date'])['Value'].sum().reset_index()



In [2]:
# Europe WHO
ItalyWHO = covidIncWHO[covidIncWHO['Area'] == 'Italy']
FranceWHO = covidIncWHO[covidIncWHO['Area'] == 'France']
GermanyWHO = covidIncWHO[covidIncWHO['Area'] == 'Germany']
SpainWHO = covidIncWHO[covidIncWHO['Area'] == 'Spain']
UnitedKingdomWHO = covidIncWHO[covidIncWHO['Area'] == 'UnitedKingdom']
SwitzerlandWHO = covidIncWHO[covidIncWHO['Area'] == 'Switzerland']

IranWHO = covidIncWHO[covidIncWHO['Area'] == 'Iran']

#Asia WHO
ChinaWHO = covidIncWHO[covidIncWHO['Area'] == 'China'].copy(deep=True)
# disabling China methodology change
#ChinaWHO.loc[China['Date'] == pd.Timestamp(2020, 2, 17), 'Value'] = np.nan

RepublicofKoreaWHO = covidIncWHO[covidIncWHO['Area'] == 'RepublicofKorea']
JapanWHO = covidIncWHO[covidIncWHO['Area'] == 'Japan']

#US WHO
UnitedStatesofAmericaWHO = covidIncWHO[covidIncWHO['Area'] == 'UnitedStatesofAmerica']

#Global-confirmed
GlobalWHO = covidIncWHO[covidIncWHO['Area'] == 'Global-confirmed'].copy(deep=True)
# disabling China methodology change
#GlobalWHO.loc[Global['Date'] == pd.Timestamp(2020, 2, 17), 'Value'] = np.nan


# Europe CSSE
ItalyCSSE = covidIncCSSE[covidIncCSSE['Country/Region'] == 'Italy']
FranceCSSE = covidIncCSSE[covidIncCSSE['Country/Region'] == 'France']
GermanyCSSE = covidIncCSSE[covidIncCSSE['Country/Region'] == 'Germany']
SpainCSSE = covidIncCSSE[covidIncCSSE['Country/Region'] == 'Spain']
UnitedKingdomCSSE = covidIncCSSE[covidIncCSSE['Country/Region'] == 'United Kingdom']
SwitzerlandCSSE = covidIncCSSE[covidIncCSSE['Country/Region'] == 'Switzerland']

IranCSSE = covidIncCSSE[covidIncCSSE['Country/Region'] == 'Iran']

#Asia CSSE
ChinaCSSE = covidIncCSSE[covidIncCSSE['Country/Region'] == 'China']
# disabling China methodology change
#ChinaWHO.loc[China['Date'] == pd.Timestamp(2020, 2, 17), 'Value'] = np.nan

RepublicofKoreaCSSE = covidIncCSSE[covidIncCSSE['Country/Region'] == 'Korea, South']
JapanCSSE = covidIncCSSE[covidIncCSSE['Country/Region'] == 'Japan']


#US CSSE

UnitedStatesofAmericaCSSE = covidIncByCountryCSSE[covidIncByCountryCSSE['Country/Region'] == 'US']

USWACSSE = covidIncCSSE[(covidIncCSSE['Country/Region'] == 'US') & (covidIncCSSE['Province/State'] == 'Washington')]
USNYCSSE = covidIncCSSE[(covidIncCSSE['Country/Region'] == 'US') & (covidIncCSSE['Province/State'] == 'New York')]
USCACSSE = covidIncCSSE[(covidIncCSSE['Country/Region'] == 'US') & (covidIncCSSE['Province/State'] == 'California')]
USNJCSSE = covidIncCSSE[(covidIncCSSE['Country/Region'] == 'US') & (covidIncCSSE['Province/State'] == 'New Jersey')]
USOthersCSSE = covidIncCSSE[
    (covidIncCSSE['Country/Region'] == 'US') & 
    ~(covidIncCSSE['Province/State'].str.contains(',')) &
    ~(covidIncCSSE['Province/State'].isin(['Washington','New York', 'California', 'New Jersey']))
].groupby('Date')['Value'].sum().reset_index()


#Global CSSE
GlobalCSSEG = covidIncAllCSSEG

In [3]:
#Europe WHO
dataEuropeWHO = [
    go.Scatter(x=ItalyWHO['Date'],y=ItalyWHO['Value'], name='Italy', mode='lines+markers'),
    go.Scatter(x=FranceWHO['Date'],y=FranceWHO['Value'], name='France', mode='lines+markers'),
    go.Scatter(x=GermanyWHO['Date'],y=GermanyWHO['Value'], name='Germany', mode='lines+markers'),
    go.Scatter(x=UnitedKingdomWHO['Date'],y=UnitedKingdomWHO['Value'], name='UnitedKingdom', mode='lines+markers'),
    go.Scatter(x=SwitzerlandWHO['Date'],y=SwitzerlandWHO['Value'], name='Switzerland', mode='lines+markers'),
    go.Scatter(x=IranWHO['Date'],y=IranWHO['Value'], name='Iran', mode='lines+markers')
]

# plot titles and axis labels
layoutEuropeWHO = go.Layout(
    title='Europe new confirmed cases, WHO',
    xaxis=dict(title='Дата'),
    yaxis=dict(title='Новые случаи за день'),
    legend_orientation="h"
)

figEuropeWHO = go.Figure(data=dataEuropeWHO, layout=layoutEuropeWHO)

#Asia WHO
dataAsiaWHO = [
    go.Scatter(x=ChinaWHO['Date'],y=ChinaWHO['Value'], name='China', mode='lines+markers'),
    go.Scatter(x=RepublicofKoreaWHO['Date'],y=RepublicofKoreaWHO['Value'], name='RepublicofKorea', mode='lines+markers'),
    go.Scatter(x=JapanWHO['Date'],y=JapanWHO['Value'], name='Japan', mode='lines+markers')
]

# plot titles and axis labels
layoutAsiaWHO = go.Layout(
    title='Asia new confirmed cases, WHO',
    xaxis=dict(title='Дата'),
    yaxis=dict(title='Новые случаи за день'),
    legend_orientation="h"
)

figAsiaWHO = go.Figure(data=dataAsiaWHO, layout=layoutAsiaWHO)



#Europe CSSE
dataEuropeCSSE = [
    go.Scatter(x=ItalyCSSE['Date'],y=ItalyCSSE['Value'], name='Italy', mode='lines+markers'),
    go.Scatter(x=FranceCSSE['Date'],y=FranceCSSE['Value'], name='France', mode='lines+markers'),
    go.Scatter(x=GermanyCSSE['Date'],y=GermanyCSSE['Value'], name='Germany', mode='lines+markers'),
    go.Scatter(x=UnitedKingdomCSSE['Date'],y=UnitedKingdomCSSE['Value'], name='UnitedKingdom', mode='lines+markers'),
    go.Scatter(x=SwitzerlandCSSE['Date'],y=SwitzerlandCSSE['Value'], name='Switzerland', mode='lines+markers'),
    go.Scatter(x=IranCSSE['Date'],y=IranCSSE['Value'], name='Iran', mode='lines+markers')
]

# plot titles and axis labels
layoutEuropeCSSE = go.Layout(
    title='Europe new confirmed cases, CSSE',
    xaxis=dict(title='Дата'),
    yaxis=dict(title='Новые случаи за день'),
    legend_orientation="h"
)

figEuropeCSSE = go.Figure(data=dataEuropeCSSE, layout=layoutEuropeCSSE)


#Asia CSSE
dataAsiaCSSE = [
    go.Scatter(x=ChinaCSSE['Date'],y=ChinaCSSE['Value'], name='China', mode='lines+markers'),
    go.Scatter(x=RepublicofKoreaCSSE['Date'],y=RepublicofKoreaCSSE['Value'], name='RepublicofKorea', mode='lines+markers'),
    go.Scatter(x=JapanCSSE['Date'],y=JapanCSSE['Value'], name='Japan', mode='lines+markers')
]

# plot titles and axis labels
layoutAsiaCSSE = go.Layout(
    title='Asia new confirmed cases, CSSE',
    xaxis=dict(title='Дата'),
    yaxis=dict(title='Новые случаи за день'),
    legend_orientation="h"
)

figAsiaCSSE = go.Figure(data=dataAsiaCSSE, layout=layoutAsiaCSSE)


#USAll
dataUSAll = [
    go.Scatter(x=UnitedStatesofAmericaWHO['Date'],y=UnitedStatesofAmericaWHO['Value'], name='UnitedStatesofAmerica', mode='lines+markers'),
    go.Scatter(x=UnitedStatesofAmericaCSSE['Date'],y=UnitedStatesofAmericaCSSE['Value'], name='UnitedStatesofAmerica CSSE', mode='lines+markers')
]

# plot titles and axis labels
layoutUSAll = go.Layout(
    title='UnitedStatesofAmerica new confirmed cases',
    xaxis=dict(title='Дата'),
    yaxis=dict(title='Новые случаи за день'),
    showlegend=True,
    legend_orientation="h"
)

figUSAll = go.Figure(data=dataUSAll, layout=layoutUSAll)


#USStates CSSE
dataUSStatesCSSE = [
    go.Scatter(x=USWACSSE['Date'],y=USWACSSE['Value'], name='WA', mode='lines+markers'),
    go.Scatter(x=USNYCSSE['Date'],y=USNYCSSE['Value'], name='NY', mode='lines+markers'),
    go.Scatter(x=USNJCSSE['Date'],y=USNJCSSE['Value'], name='NJ', mode='lines+markers'),
    go.Scatter(x=USCACSSE['Date'],y=USCACSSE['Value'], name='CA', mode='lines+markers'),
    go.Scatter(x=USOthersCSSE['Date'],y=USOthersCSSE['Value'], name='others', mode='lines+markers')
]

# plot titles and axis labels
layoutUSStatesCSSE = go.Layout(
    title='UnitedStatesofAmerica new confirmed cases by states',
    xaxis=dict(title='Дата'),
    yaxis=dict(title='Новые случаи за день'),
    showlegend=True,
    legend_orientation="h"
)

figUSStatesCSSE = go.Figure(data=dataUSStatesCSSE, layout=layoutUSStatesCSSE)




#Global
dataGlobalAll = [
    go.Scatter(x=GlobalWHO['Date'],y=GlobalWHO['Value'], name='Global WHO', mode='lines+markers'),
    go.Scatter(x=GlobalCSSEG['Date'],y=GlobalCSSEG['Value'], name='Global CSSE', mode='lines+markers')
]

# plot titles and axis labels
layoutGlobalAll = go.Layout(
    title='Global new confirmed cases',
    xaxis=dict(title='Дата'),
    yaxis=dict(title='Новые случаи за день'),
    showlegend=True,
    legend_orientation="h"
)

figGlobalAll = go.Figure(data=dataGlobalAll, layout=layoutGlobalAll)




iplot(figEuropeWHO)
iplot(figEuropeCSSE)
iplot(figAsiaWHO)
iplot(figAsiaCSSE)
iplot(figUSAll)
iplot(figUSStatesCSSE)
iplot(figGlobalAll)

